# Combining Registry, License, and General Funds Data
Using Fuzz + TF-IDF

* Reigstry: all registered businesses
* License: Opendata License
* General Funds: all payment history recorded by OEO

In [1]:
import mwdsbe
import mwdsbe.datasets.licenses as licenses
import schuylkill as skool
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import time

## Merge Registry and License

In [2]:
registry = mwdsbe.load_registry() # geopandas df
license = licenses.CommercialActivityLicenses().get()

In [3]:
len(registry)

3119

In [5]:
len(license)

203578

In [6]:
# clean data
ignore_words = ['inc', 'group', 'llc', 'corp', 'pc', 'incorporated', 'ltd', 'co', 'associates', 'services', 'company', 'enterprises', 'enterprise', 'service', 'corporation']
cleaned_registry = skool.clean_strings(registry, ['company_name', 'dba_name'], True, ignore_words)
cleaned_license = skool.clean_strings(license, ['company_name'], True, ignore_words)

cleaned_registry = cleaned_registry.dropna(subset=['company_name'])
cleaned_license = cleaned_license.dropna(subset=['company_name'])

In [7]:
t1 = time.time()
merged = (
    skool.fuzzy_merge(cleaned_registry, cleaned_license, on="company_name", score_cutoff=95)
    .pipe(skool.fuzzy_merge, cleaned_registry, cleaned_license, left_on="dba_name", right_on="company_name", score_cutoff=95)
    .pipe(skool.tf_idf_merge, cleaned_registry, cleaned_license, on="company_name", score_cutoff=85)
    .pipe(skool.tf_idf_merge, cleaned_registry, cleaned_license, left_on="dba_name", right_on="company_name", score_cutoff=85)
)
t = time.time() - t1

In [8]:
print('Execution time:', t/60, 'min')

Execution time: 30.124040285746258 min


In [9]:
matched = merged.dropna(subset=['company_name_y'])

In [10]:
len(matched)

1566

In [11]:
matched.head()

,company_name_x,dba_name,owner_first,owner_last,location,location_city,location_state,zip_code,mailing_address,mailing_city,...,lat,lng,geometry,in_philly_msa,match_probability,right_index,license_num,issue_date,license_status,company_name_y
0,119 degrees architects,NaN,Rafael,Utrera,"1503 Green Street, Suite # 4",Philadelphia,PA,19130.0,"1503 Green Street, Suite # 4",Philadelphia,...,39.964275,-75.163042,POINT (-75.16304 39.96427),True,1.0,131105.0,480115.0,2009-07-31 00:00:00+00:00,Active,119 degrees architects
5,22,NaN,Caroline,Harper,757 Public Road,Bethlehem,PA,18015.0,757 Public Road,Bethlehem,...,40.599276,-75.408425,POINT (-75.40842 40.59928),False,1.0,168732.0,667661.0,2015-06-10 00:00:00+00:00,Active,22
7,24 hour cleaning,NaN,Mary Colleen,Zoltowski,14005 Barcalow Street,Philadelphia,PA,19116.0,14005 Barcalow Street,Philadelphia,...,40.131349,-75.014284,POINT (-75.01428 40.13135),True,1.0,17320.0,120711.0,1999-08-30 00:00:00+00:00,Active,24 hour cleaning
8,259 strategies,NaN,Chaka,"Fattah, Jr.","Two Logan Square, Suite 1900",Philadelphia,PA,19103.0,"Two Logan Square, Suite 1900",Philadelphia,...,39.955726,-75.169784,POINT (-75.16978 39.95573),True,1.0,102422.0,346817.0,2005-11-28 00:00:00+00:00,Active,259 strategies
12,4u,stellar,Liang,Chen,"70 West 36th Street, Ste. #702",New York,NY,10018.0,"70 West 36th Street, Ste. #702",New York,...,40.750587,-73.986278,POINT (-73.98628 40.75059),False,1.0,141063.0,531189.0,2011-04-22 00:00:00+00:00,Active,4u


In [28]:
# add original company name before cleaning
company_names = registry[['company_name', 'dba_name']]
matches = company_names.merge(matched, left_index=True, right_index=True)

In [30]:
matches

,company_name,dba_name_x,company_name_x,dba_name_y,owner_first,owner_last,location,location_city,location_state,zip_code,...,lat,lng,geometry,in_philly_msa,match_probability,right_index,license_num,issue_date,license_status,company_name_y
0,119 Degrees Architects,NaN,119 degrees architects,NaN,Rafael,Utrera,"1503 Green Street, Suite # 4",Philadelphia,PA,19130.0,...,39.964275,-75.163042,POINT (-75.16304 39.96427),True,1.00000,131105.0,480115.0,2009-07-31 00:00:00+00:00,Active,119 degrees architects
5,"22 Group, LLC",NaN,22,NaN,Caroline,Harper,757 Public Road,Bethlehem,PA,18015.0,...,40.599276,-75.408425,POINT (-75.40842 40.59928),False,1.00000,168732.0,667661.0,2015-06-10 00:00:00+00:00,Active,22
7,24 Hour Cleaning Services Ltd.,NaN,24 hour cleaning,NaN,Mary Colleen,Zoltowski,14005 Barcalow Street,Philadelphia,PA,19116.0,...,40.131349,-75.014284,POINT (-75.01428 40.13135),True,1.00000,17320.0,120711.0,1999-08-30 00:00:00+00:00,Active,24 hour cleaning
8,"259 Strategies, LLC",NaN,259 strategies,NaN,Chaka,"Fattah, Jr.","Two Logan Square, Suite 1900",Philadelphia,PA,19103.0,...,39.955726,-75.169784,POINT (-75.16978 39.95573),True,1.00000,102422.0,346817.0,2005-11-28 00:00:00+00:00,Active,259 strategies
12,4U Services Inc.,"Stellar Services, Inc",4u,stellar,Liang,Chen,"70 West 36th Street, Ste. #702",New York,NY,10018.0,...,40.750587,-73.986278,POINT (-73.98628 40.75059),False,1.00000,141063.0,531189.0,2011-04-22 00:00:00+00:00,Active,4u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3107,"Zavorski Masonry Restoration, Inc.",NaN,zavorski masonry restoration,NaN,Lisa,Zavorski,717 Ford Avenue,Langhorne,PA,19047.0,...,40.139511,-74.895792,POINT (-74.89579 40.13951),True,1.00000,101652.0,343160.0,2005-10-31 00:00:00+00:00,Active,zavorski masonry restoration
3110,"Zenga Engineering, LLC",NaN,zenga engineering,NaN,Gwendolyn,Lodise,313 East Broad Street,Palmyra,NJ,8065.0,...,40.006190,-75.018174,POINT (-75.01817 40.00619),True,1.00000,136814.0,509641.0,2010-07-26 00:00:00+00:00,Active,zenga engineering
3113,"Zizza Highway Services, Inc.",NaN,zizza highway,NaN,Arlene,Zizza,382 CONCHESTER HIGHWAY,Glen Mills,PA,19342.0,...,39.873722,-75.504880,POINT (-75.50488 39.87372),True,1.00000,30230.0,161685.0,2003-05-09 00:00:00+00:00,Active,zizza highway
3115,"ZONES, LLC",NaN,zones,NaN,Mr.,Lalji,1102 15TH ST SW,Auburn,WA,98001.0,...,47.295612,-122.246186,POINT (-122.24619 47.29561),False,0.86094,26525.0,145187.0,2002-05-06 00:00:00+00:00,Active,zone


In [31]:
matches.drop(['company_name_x', 'dba_name_y'], axis=1, inplace=True)

In [34]:
matches.rename(columns={"dba_name_x": "dba_name"}, inplace=True)

In [36]:
# save matched data btw registry and license
matches.to_csv(r'C:\Users\dabinlee\Documents\GitHub\mwdsbe_binny\MWDSBE\analysis\data\processed-data\Registry-OpendataLicense\fuzz95_tfidf85.csv', header=True)

## Merge Registry and General Fund

In [38]:
gf = pd.read_excel(r'C:\Users\dabinlee\Documents\GitHub\mwdsbe_binny\MWDSBE\analysis\data\downloaded-data\general-funds\cwedp_37_report.xlsx', sheet_name='general_funds')

In [39]:
# filter gf with unrevelant maj_class and na vendor name
gf = gf.loc[gf['MAJ_CLASS'] != 1]
gf = gf.loc[gf['VEND_NAME'].dropna().index]
len(gf)

243375

In [40]:
# clean data
ignore_words = ['inc', 'group', 'llc', 'corp', 'pc', 'incorporated', 'ltd', 'co', 'associates', 'services', 'company', 'enterprises', 'enterprise', 'service', 'corporation']
cleaned_gf = skool.clean_strings(gf, ['VEND_NAME'], True, ignore_words)
cleaned_gf = cleaned_gf.dropna(subset=['VEND_NAME'])

In [41]:
t1 = time.time()
merged2 = (
    skool.fuzzy_merge(cleaned_registry, cleaned_gf, left_on="company_name", right_on="VEND_NAME", score_cutoff=95)
    .pipe(skool.fuzzy_merge, cleaned_registry, cleaned_gf, left_on="dba_name", right_on="VEND_NAME", score_cutoff=95)
    .pipe(skool.tf_idf_merge, cleaned_registry, cleaned_gf, left_on="company_name", right_on="VEND_NAME", score_cutoff=80)
    .pipe(skool.tf_idf_merge, cleaned_registry, cleaned_gf, left_on="dba_name", right_on="VEND_NAME", score_cutoff=80)
)
t = time.time() - t1

In [43]:
print('Execution time:', t/60, 'min')

Execution time: 22.05843669573466 min


In [42]:
matched2 = merged2.dropna(subset=['VEND_NAME'])

In [44]:
len(matched2)

150

In [45]:
# Manually remove 14 wrong matches
wrong_matches = ['a g consulting engineering', 'amercom consulting engineers', 'awk consulting engineers', 
                 'briones consulting engineering', 'dy consulting engineer p c', 'environmental management',
                 'events unlimited', 'hill environmental', 'kma consulting engineers', 'milligan and', 
                 'ms technologies', 'specialists', 'site contractors', 'stell environmental']
matches2 = matched2.loc[matched2['company_name'].apply(lambda x : x not in wrong_matches)]
len(matches2)

136

In [46]:
# get full payments per vendor
unique_vendors = matches2['VEND_NAME'].tolist()
full_matches = cleaned_gf.loc[cleaned_gf['VEND_NAME'].apply(lambda x : x in unique_vendors)]

In [50]:
data = matches2.merge(full_matches, on='VEND_NAME')

In [53]:
full_data = company_names.merge(matches2, left_index=True, right_index=True)

In [54]:
full_data

,company_name_x,dba_name_x,company_name_y,dba_name_y,owner_first,owner_last,location,location_city,location_state,zip_code,...,MAJ_CLASS,CLASS,DOC_NUM,TRANS_DESC,DOC_REF_NUM,VEND_NAME,VEND_NUM,AMT,DOC_NUM4,DOC_REF4
12,4U Services Inc.,"Stellar Services, Inc",4u,stellar,Liang,Chen,"70 West 36th Street, Ste. #702",New York,NY,10018.0,...,2.0,0250,PVMD1902159601,*1669* MAY19 DAO,MDXX1900001301,4u,113189413 02,7420.00,PVMD,MDXX
18,A & A Court Reporting,Andrea St. John,a a court reporting,andrea st john,Andrea,St. John,"303 Chestnut Street, Second Floor",Philadelphia,PA,19106.0,...,2.0,0258,PVXX1901098501,*35987* APR19 JAMES MCINTYRE V. CITY,NaN,a a court reporting,233034758 02,350.05,PVXX,NaN
19,A & A Glove & Safety Company,NaN,a a glove safety,NaN,Ana M,Fisher,20 Richey Avenue,Collingswood,NJ,8107.0,...,3.0,0316,VCXX1903158301,*74033* ELEC CHAIN HOIST AUG18,POXX1910343701,a a glove safety,223525598 01,2254.00,VCXX,POXX
60,"Acacia Financial Group, Inc.",NaN,acacia financial,NaN,Noreen,White,"6000 Midlantic Drive, Suite 410 North",Mt. Laurel,NJ,8054.0,...,7.0,0705,PVXX1901010901,*09B EXTENSION* FA FEES,NaN,acacia financial,204775407 01,3000.00,PVXX,NaN
65,ACCLAIM SYSTEMS INC,NaN,acclaim systems,NaN,Kailash,Kalantri,110 EAST PENNSYLVANIA BOULEVARD,Feasterville,PA,19053.0,...,2.0,0251,PVMD1901613001,*PHL13* PLIS SUPPT/MAINT 4/2018-3/2019 MAR19,MDXX1800135401,acclaim systems,364182176 02,91393.34,PVMD,MDXX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3052,"West Chester Environmental, LLC",NaN,west chester environmental,NaN,Matthew,Abraham,1248 Wrights Ln,West Chester,PA,19380.0,...,2.0,0250,PVMD1901826802,*5774* 2522 N COLORADO ST APR19,MDXX1800025901,westchester environmental,364510971 01,296.50,PVMD,MDXX
3056,"WFGD Studio, LLC",NaN,wfgd studio,NaN,Marcella,Coffey,"718 Arch Street, Suite 302s",Philadelphia,PA,19106.0,...,2.0,0250,PVMD1900042901,*5765*CONTRACT MPXX18000282 FOR MAY-JUN 2018 SVCS,MPXX1800028201,wfgd studio,454869723 01,8295.00,PVMD,MPXX
3097,"YIKES, Inc.",NaN,yikes,NaN,Tracy,Levesque,204 East Girard Avenue,Philadelphia,PA,19125.0,...,2.0,0250,PVMD1900395301,*9126* PROF SERVICES RENDERED,MPXX1900018701,yikes,232940295 01,18788.00,PVMD,MPXX
3106,"Zanaras Reporting & Video, LLC",NaN,zanaras reporting video,NaN,JULIE,ZANARAS,"1845 WALNUT STREET, Suite 938",Philadelphia,PA,19103.0,...,2.0,0258,PVXX1900785401,*56215* NOV18 99185 STREETS HINTON V. SEPTA,NaN,zanaras reporting video,510620006 01,231.80,PVXX,NaN


In [55]:
payments = full_matches.groupby('VEND_NAME')['AMT'].sum().to_frame('TOTAL_AMT').reset_index()

In [56]:
combined = full_data.merge(payments, on='VEND_NAME')

In [57]:
combined

,company_name_x,dba_name_x,company_name_y,dba_name_y,owner_first,owner_last,location,location_city,location_state,zip_code,...,CLASS,DOC_NUM,TRANS_DESC,DOC_REF_NUM,VEND_NAME,VEND_NUM,AMT,DOC_NUM4,DOC_REF4,TOTAL_AMT
0,4U Services Inc.,"Stellar Services, Inc",4u,stellar,Liang,Chen,"70 West 36th Street, Ste. #702",New York,NY,10018.0,...,0250,PVMD1902159601,*1669* MAY19 DAO,MDXX1900001301,4u,113189413 02,7420.00,PVMD,MDXX,178240.00
1,A & A Court Reporting,Andrea St. John,a a court reporting,andrea st john,Andrea,St. John,"303 Chestnut Street, Second Floor",Philadelphia,PA,19106.0,...,0258,PVXX1901098501,*35987* APR19 JAMES MCINTYRE V. CITY,NaN,a a court reporting,233034758 02,350.05,PVXX,NaN,2607.50
2,Court Reporting Associates,NaN,court reporting,NaN,Margarita,Tsouderos,P.O. Box 515 - Apt. #201,Wynnewood,PA,19096.0,...,0258,PVXX1900741601,*35642* JAN19 KENNETH GRIMES V. CITY,NaN,a a court reporting,233034758 02,586.00,PVXX,NaN,2607.50
3,A & A Glove & Safety Company,NaN,a a glove safety,NaN,Ana M,Fisher,20 Richey Avenue,Collingswood,NJ,8107.0,...,0316,VCXX1903158301,*74033* ELEC CHAIN HOIST AUG18,POXX1910343701,a a glove safety,223525598 01,2254.00,VCXX,POXX,45686.49
4,"Acacia Financial Group, Inc.",NaN,acacia financial,NaN,Noreen,White,"6000 Midlantic Drive, Suite 410 North",Mt. Laurel,NJ,8054.0,...,0705,PVXX1901010901,*09B EXTENSION* FA FEES,NaN,acacia financial,204775407 01,3000.00,PVXX,NaN,3000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,"West Chester Environmental, LLC",NaN,west chester environmental,NaN,Matthew,Abraham,1248 Wrights Ln,West Chester,PA,19380.0,...,0250,PVMD1901826802,*5774* 2522 N COLORADO ST APR19,MDXX1800025901,westchester environmental,364510971 01,296.50,PVMD,MDXX,28206.75
132,"WFGD Studio, LLC",NaN,wfgd studio,NaN,Marcella,Coffey,"718 Arch Street, Suite 302s",Philadelphia,PA,19106.0,...,0250,PVMD1900042901,*5765*CONTRACT MPXX18000282 FOR MAY-JUN 2018 SVCS,MPXX1800028201,wfgd studio,454869723 01,8295.00,PVMD,MPXX,41375.00
133,"YIKES, Inc.",NaN,yikes,NaN,Tracy,Levesque,204 East Girard Avenue,Philadelphia,PA,19125.0,...,0250,PVMD1900395301,*9126* PROF SERVICES RENDERED,MPXX1900018701,yikes,232940295 01,18788.00,PVMD,MPXX,31528.00
134,"Zanaras Reporting & Video, LLC",NaN,zanaras reporting video,NaN,JULIE,ZANARAS,"1845 WALNUT STREET, Suite 938",Philadelphia,PA,19103.0,...,0258,PVXX1900785401,*56215* NOV18 99185 STREETS HINTON V. SEPTA,NaN,zanaras reporting video,510620006 01,231.80,PVXX,NaN,4588.10


In [58]:
combined.drop(["company_name_y", "dba_name_y", "right_index", "AMT"], axis=1, inplace=True)

In [59]:
combined

,company_name_x,dba_name_x,owner_first,owner_last,location,location_city,location_state,zip_code,mailing_address,mailing_city,...,MAJ_CLASS,CLASS,DOC_NUM,TRANS_DESC,DOC_REF_NUM,VEND_NAME,VEND_NUM,DOC_NUM4,DOC_REF4,TOTAL_AMT
0,4U Services Inc.,"Stellar Services, Inc",Liang,Chen,"70 West 36th Street, Ste. #702",New York,NY,10018.0,"70 West 36th Street, Ste. #702",New York,...,2.0,0250,PVMD1902159601,*1669* MAY19 DAO,MDXX1900001301,4u,113189413 02,PVMD,MDXX,178240.00
1,A & A Court Reporting,Andrea St. John,Andrea,St. John,"303 Chestnut Street, Second Floor",Philadelphia,PA,19106.0,"P.O. Box 74, Suite 4",Moorestown,...,2.0,0258,PVXX1901098501,*35987* APR19 JAMES MCINTYRE V. CITY,NaN,a a court reporting,233034758 02,PVXX,NaN,2607.50
2,Court Reporting Associates,NaN,Margarita,Tsouderos,P.O. Box 515 - Apt. #201,Wynnewood,PA,19096.0,P.O. Box 515 - Apt. #201,Wynnewood,...,2.0,0258,PVXX1900741601,*35642* JAN19 KENNETH GRIMES V. CITY,NaN,a a court reporting,233034758 02,PVXX,NaN,2607.50
3,A & A Glove & Safety Company,NaN,Ana M,Fisher,20 Richey Avenue,Collingswood,NJ,8107.0,20 Richey Avenue,Collingswood,...,3.0,0316,VCXX1903158301,*74033* ELEC CHAIN HOIST AUG18,POXX1910343701,a a glove safety,223525598 01,VCXX,POXX,45686.49
4,"Acacia Financial Group, Inc.",NaN,Noreen,White,"6000 Midlantic Drive, Suite 410 North",Mt. Laurel,NJ,8054.0,"6000 Midlantic Drive, Suite 410 North",Mt. Laurel,...,7.0,0705,PVXX1901010901,*09B EXTENSION* FA FEES,NaN,acacia financial,204775407 01,PVXX,NaN,3000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,"West Chester Environmental, LLC",NaN,Matthew,Abraham,1248 Wrights Ln,West Chester,PA,19380.0,1248 Wrights Ln,West Chester,...,2.0,0250,PVMD1901826802,*5774* 2522 N COLORADO ST APR19,MDXX1800025901,westchester environmental,364510971 01,PVMD,MDXX,28206.75
132,"WFGD Studio, LLC",NaN,Marcella,Coffey,"718 Arch Street, Suite 302s",Philadelphia,PA,19106.0,"718 Arch Street, Suite 302S",Philadelphia,...,2.0,0250,PVMD1900042901,*5765*CONTRACT MPXX18000282 FOR MAY-JUN 2018 SVCS,MPXX1800028201,wfgd studio,454869723 01,PVMD,MPXX,41375.00
133,"YIKES, Inc.",NaN,Tracy,Levesque,204 East Girard Avenue,Philadelphia,PA,19125.0,204 East Girard Avenue,Philadelphia,...,2.0,0250,PVMD1900395301,*9126* PROF SERVICES RENDERED,MPXX1900018701,yikes,232940295 01,PVMD,MPXX,31528.00
134,"Zanaras Reporting & Video, LLC",NaN,JULIE,ZANARAS,"1845 WALNUT STREET, Suite 938",Philadelphia,PA,19103.0,"1845 WALNUT STREET, Suite 938",Philadelphia,...,2.0,0258,PVXX1900785401,*56215* NOV18 99185 STREETS HINTON V. SEPTA,NaN,zanaras reporting video,510620006 01,PVXX,NaN,4588.10


In [62]:
combined.rename(columns={"company_name_x": "company_name", "dba_name_x": "dba_name"}, inplace=True)

In [70]:
combined

,company_name,dba_name,owner_first,owner_last,location,location_city,location_state,zip_code,mailing_address,mailing_city,...,MAJ_CLASS,CLASS,DOC_NUM,TRANS_DESC,DOC_REF_NUM,VEND_NAME,VEND_NUM,DOC_NUM4,DOC_REF4,TOTAL_AMT
0,4U Services Inc.,"Stellar Services, Inc",Liang,Chen,"70 West 36th Street, Ste. #702",New York,NY,10018.0,"70 West 36th Street, Ste. #702",New York,...,2.0,0250,PVMD1902159601,*1669* MAY19 DAO,MDXX1900001301,4u,113189413 02,PVMD,MDXX,178240.00
1,A & A Court Reporting,Andrea St. John,Andrea,St. John,"303 Chestnut Street, Second Floor",Philadelphia,PA,19106.0,"P.O. Box 74, Suite 4",Moorestown,...,2.0,0258,PVXX1901098501,*35987* APR19 JAMES MCINTYRE V. CITY,NaN,a a court reporting,233034758 02,PVXX,NaN,2607.50
2,Court Reporting Associates,NaN,Margarita,Tsouderos,P.O. Box 515 - Apt. #201,Wynnewood,PA,19096.0,P.O. Box 515 - Apt. #201,Wynnewood,...,2.0,0258,PVXX1900741601,*35642* JAN19 KENNETH GRIMES V. CITY,NaN,a a court reporting,233034758 02,PVXX,NaN,2607.50
3,A & A Glove & Safety Company,NaN,Ana M,Fisher,20 Richey Avenue,Collingswood,NJ,8107.0,20 Richey Avenue,Collingswood,...,3.0,0316,VCXX1903158301,*74033* ELEC CHAIN HOIST AUG18,POXX1910343701,a a glove safety,223525598 01,VCXX,POXX,45686.49
4,"Acacia Financial Group, Inc.",NaN,Noreen,White,"6000 Midlantic Drive, Suite 410 North",Mt. Laurel,NJ,8054.0,"6000 Midlantic Drive, Suite 410 North",Mt. Laurel,...,7.0,0705,PVXX1901010901,*09B EXTENSION* FA FEES,NaN,acacia financial,204775407 01,PVXX,NaN,3000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,"West Chester Environmental, LLC",NaN,Matthew,Abraham,1248 Wrights Ln,West Chester,PA,19380.0,1248 Wrights Ln,West Chester,...,2.0,0250,PVMD1901826802,*5774* 2522 N COLORADO ST APR19,MDXX1800025901,westchester environmental,364510971 01,PVMD,MDXX,28206.75
132,"WFGD Studio, LLC",NaN,Marcella,Coffey,"718 Arch Street, Suite 302s",Philadelphia,PA,19106.0,"718 Arch Street, Suite 302S",Philadelphia,...,2.0,0250,PVMD1900042901,*5765*CONTRACT MPXX18000282 FOR MAY-JUN 2018 SVCS,MPXX1800028201,wfgd studio,454869723 01,PVMD,MPXX,41375.00
133,"YIKES, Inc.",NaN,Tracy,Levesque,204 East Girard Avenue,Philadelphia,PA,19125.0,204 East Girard Avenue,Philadelphia,...,2.0,0250,PVMD1900395301,*9126* PROF SERVICES RENDERED,MPXX1900018701,yikes,232940295 01,PVMD,MPXX,31528.00
134,"Zanaras Reporting & Video, LLC",NaN,JULIE,ZANARAS,"1845 WALNUT STREET, Suite 938",Philadelphia,PA,19103.0,"1845 WALNUT STREET, Suite 938",Philadelphia,...,2.0,0258,PVXX1900785401,*56215* NOV18 99185 STREETS HINTON V. SEPTA,NaN,zanaras reporting video,510620006 01,PVXX,NaN,4588.10


In [77]:
# save
combined.to_csv(r'C:\Users\dabinlee\Documents\GitHub\mwdsbe_binny\MWDSBE\analysis\data\processed-data\Registry-GeneralFunds\joined-136-total-payments.csv', header=True)

## Merge combined registry-license and registry-general funds